In [ ]:
# ==============================================================================
# SECCIÓN 0: IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN INICIAL
# ==============================================================================
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import warnings

from darts import TimeSeries
from darts.models import TCNModel, RNNModel
from darts.metrics import rmse
from darts.dataprocessing.transformers import Scaler

# Ignorar advertencias para una salida más limpia
warnings.filterwarnings('ignore')

# Configuración de Matplotlib para los gráficos
plt.style.use('fivethirtyeight')

print("Librerías importadas correctamente.")

# ==============================================================================
# SECCIÓN 1: ANÁLISIS EXPLORATORIO DE DATOS (EDA) Y PREPARACIÓN
# ==============================================================================
print("\n--- Iniciando Sección 1: Análisis Exploratorio de Datos ---")

# --- 1.1 Carga de Datos ---
# NOTA: Asegúrate de que la ruta a tus archivos sea la correcta.
# El usuario especificó "../../Datos/", ajusta la variable 'path' si es necesario.
try:
    path = "./" # Se asume que los archivos están en el mismo directorio del notebook.
                 # Cambia a "../../Datos/" si esa es tu estructura de carpetas.
    df_train = pd.read_csv(f'{path}df_train.csv')
    df_test = pd.read_csv(f'{path}df_test.csv')
    sample_submission = pd.read_csv(f'{path}sample_submission.csv')
except FileNotFoundError:
    print("Error: No se encontraron los archivos CSV. Por favor, verifica la ruta en la variable 'path'.")
    # Salir si los archivos no se encuentran
    exit()


# --- 1.2 Preprocesamiento y Limpieza de Datos ---
# Función para preprocesar los dataframes
def preprocess_data(df):
    # Asumiendo que las columnas se llaman 'FECHA' y 'UNIDADES_VENDIDAS'
    # Renombrar columnas para estandarizar
    df = df.rename(columns={
        'FECHA': 'fecha',
        'UNIDADES_VENDIDAS': 'unidades_vendidas'
    })
    # Convertir la columna 'fecha' a formato datetime
    df['fecha'] = pd.to_datetime(df['fecha'])
    # Agrupar por semana para asegurar una frecuencia constante
    # Se suma las unidades vendidas por semana
    df = df.set_index('fecha').resample('W-MON').sum().reset_index()
    return df

df_train = preprocess_data(df_train)

print("Preprocesamiento inicial completado.")
print("Primeras 5 filas del set de entrenamiento:")
print(df_train.head())
print("\nÚltimas 5 filas del set de entrenamiento:")
print(df_train.tail())

# --- 1.3 Creación de la Serie de Tiempo con Darts ---
# Darts requiere un objeto TimeSeries. Usaremos 'fecha' como el índice de tiempo
# y 'unidades_vendidas' como el valor.
full_series = TimeSeries.from_dataframe(
    df_train,
    time_col='fecha',
    value_cols='unidades_vendidas',
    freq='W-MON' # Frecuencia semanal, comenzando en lunes
)

# --- 1.4 Visualización de la Serie de Tiempo Completa ---
plt.figure(figsize=(14, 7))
full_series.plot(label='Ventas históricas')
plt.title('Serie de Tiempo Completa de Unidades Vendidas')
plt.xlabel('Fecha')
plt.ylabel('Unidades Vendidas')
plt.legend()
plt.show()

# --- 1.5 División en Sets de Entrenamiento y Validación ---
# Entrenamiento: datos con año < 2021
# Validación: datos con año >= 2021
train_series, val_series = full_series.split_before(pd.Timestamp('20210101'))

print(f"\nFechas de entrenamiento: de {train_series.start_time()} a {train_series.end_time()}")
print(f"Fechas de validación: de {val_series.start_time()} a {val_series.end_time()}")

# Visualización de la división
plt.figure(figsize=(14, 7))
train_series.plot(label='Entrenamiento (train)')
val_series.plot(label='Validación (validation)')
plt.title('División en Datos de Entrenamiento y Validación')
plt.xlabel('Fecha')
plt.ylabel('Unidades Vendidas')
plt.legend()
plt.show()

# --- 1.6 Normalización de los Datos ---
# Escalar los datos es una buena práctica para redes neuronales.
# Se ajusta el escalador SÓLO con los datos de entrenamiento para evitar fuga de datos.
scaler = Scaler()
train_series_scaled = scaler.fit_transform(train_series)
val_series_scaled = scaler.transform(val_series)
full_series_scaled = scaler.transform(full_series)

print("Datos normalizados para el entrenamiento de modelos.")
print("--- Fin Sección 1 ---")


# ==============================================================================
# SECCIÓN 2: IMPLEMENTACIÓN Y COMPARACIÓN DE MODELOS (TCN, LSTM, GRU)
# ==============================================================================
print("\n--- Iniciando Sección 2: Implementación y Selección de Modelos ---")

# --- 2.1 Definición de Parámetros Comunes para los Modelos ---
# Estos parámetros se eligen de forma empírica. Para un proyecto real, se deberían optimizar.
INPUT_CHUNK_LENGTH = 52  # Usar las últimas 52 semanas (1 año) para predecir
OUTPUT_CHUNK_LENGTH = 12 # Predecir 12 semanas hacia adelante
N_EPOCHS = 100           # Número de épocas para entrenar
BATCH_SIZE = 32

# Parámetros para el optimizador de Pytorch
torch_optimizer_kwargs = {
    "lr": 1e-3, # Tasa de aprendizaje
}

# Configuración para la reproducibilidad
torch.manual_seed(42)
np.random.seed(42)

# --- 2.2 Creación y Entrenamiento de Modelos ---
# Se crea un diccionario para almacenar los modelos y sus pronósticos
models = {}
predictions = {}
losses = {}

# Modelo 1: Temporal Convolutional Network (TCN)
print("\nEntrenando modelo TCN...")
model_tcn = TCNModel(
    input_chunk_length=INPUT_CHUNK_LENGTH,
    output_chunk_length=OUTPUT_CHUNK_LENGTH,
    n_epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    dropout=0.1,
    dilation_base=2,
    weight_norm=True,
    kernel_size=5,
    num_filters=3,
    random_state=42,
    pl_trainer_kwargs={"accelerator": "auto"} # Usa GPU si está disponible
)
model_tcn.fit(train_series_scaled, verbose=False)
pred_tcn_scaled = model_tcn.predict(n=len(val_series))
pred_tcn = scaler.inverse_transform(pred_tcn_scaled) # Re-escalar a valores originales
losses['TCN'] = rmse(val_series, pred_tcn)
models['TCN'] = model_tcn
predictions['TCN'] = pred_tcn

# Modelo 2: Long Short-Term Memory (LSTM)
print("Entrenando modelo LSTM...")
model_lstm = RNNModel(
    model='LSTM',
    input_chunk_length=INPUT_CHUNK_LENGTH,
    output_chunk_length=OUTPUT_CHUNK_LENGTH,
    hidden_dim=25,
    n_rnn_layers=2,
    dropout=0.1,
    n_epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    optimizer_kwargs=torch_optimizer_kwargs,
    random_state=42,
    pl_trainer_kwargs={"accelerator": "auto"}
)
model_lstm.fit(train_series_scaled, verbose=False)
pred_lstm_scaled = model_lstm.predict(n=len(val_series))
pred_lstm = scaler.inverse_transform(pred_lstm_scaled)
losses['LSTM'] = rmse(val_series, pred_lstm)
models['LSTM'] = model_lstm
predictions['LSTM'] = pred_lstm

# Modelo 3: Gated Recurrent Unit (GRU)
print("Entrenando modelo GRU...")
model_gru = RNNModel(
    model='GRU',
    input_chunk_length=INPUT_CHUNK_LENGTH,
    output_chunk_length=OUTPUT_CHUNK_LENGTH,
    hidden_dim=25,
    n_rnn_layers=2,
    dropout=0.1,
    n_epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    optimizer_kwargs=torch_optimizer_kwargs,
    random_state=42,
    pl_trainer_kwargs={"accelerator": "auto"}
)
model_gru.fit(train_series_scaled, verbose=False)
pred_gru_scaled = model_gru.predict(n=len(val_series))
pred_gru = scaler.inverse_transform(pred_gru_scaled)
losses['GRU'] = rmse(val_series, pred_gru)
models['GRU'] = model_gru
predictions['GRU'] = pred_gru


# --- 2.3 Comparación de Resultados ---
print("\n--- Comparación de Modelos en el Set de Validación ---")
for model_name, loss in losses.items():
    print(f"Loss (RMSE) del modelo {model_name}: {loss:.4f}")

# Encontrar el mejor modelo
best_model_name = min(losses, key=losses.get)
best_model_loss = losses[best_model_name]
print(f"\nEl mejor modelo es '{best_model_name}' con un RMSE de {best_model_loss:.4f}")

# --- 2.4 Visualización de los Pronósticos de Validación ---
plt.figure(figsize=(14, 8))
val_series.plot(label='Valores Reales (Validación)', lw=3)

for model_name, pred in predictions.items():
    pred.plot(label=f'Pronóstico {model_name}')

plt.title('Comparación de Pronósticos en el Set de Validación')
plt.xlabel('Fecha')
plt.ylabel('Unidades Vendidas')
plt.legend()
plt.show()

print("--- Fin Sección 2 ---")


# ==============================================================================
# SECCIÓN 3: ENTRENAMIENTO FINAL Y GENERACIÓN DE PRONÓSTICOS PARA 2022
# ==============================================================================
print("\n--- Iniciando Sección 3: Entrenamiento Final y Pronóstico ---")

# --- 3.1 Preparación del Dataset Completo ---
# Combinar df_train y df_test
df_test = preprocess_data(df_test)
df_full = pd.concat([df_train, df_test], ignore_index=True)

print("Datos de entrenamiento y prueba combinados.")
print(f"Total de registros para el entrenamiento final: {len(df_full)}")

# Crear la serie de tiempo completa para el re-entrenamiento
final_series = TimeSeries.from_dataframe(
    df_full,
    time_col='fecha',
    value_cols='unidades_vendidas',
    freq='W-MON'
)

# Normalizar la serie de tiempo completa
final_scaler = Scaler()
final_series_scaled = final_scaler.fit_transform(final_series)


# --- 3.2 Re-entrenamiento del Mejor Modelo ---
print(f"\nRe-entrenando el mejor modelo ('{best_model_name}') con todos los datos...")

# Obtener la arquitectura del mejor modelo
best_model_architecture = models[best_model_name]

# Entrenar el modelo en el dataset completo
best_model_architecture.fit(final_series_scaled, verbose=False)

print("Re-entrenamiento completado.")

# --- 3.3 Generación del Pronóstico para 2022 ---
# Determinar el número de semanas a predecir en 2022
# El archivo sample_submission indica las semanas que necesitamos
# Vamos a calcular cuántas semanas hay desde el final de nuestros datos hasta fin de 2022.
last_date_in_data = final_series.end_time()
forecast_end_date = pd.Timestamp('2022-12-31')
# Darts predice el número de pasos (semanas en este caso)
n_forecast_steps = (forecast_end_date.to_period('W-MON') - last_date_in_data.to_period('W-MON')).n

print(f"Última fecha en los datos: {last_date_in_data.strftime('%Y-%m-%d')}")
print(f"Se pronosticarán {n_forecast_steps} semanas para cubrir hasta finales de 2022.")

# Realizar el pronóstico
final_forecast_scaled = best_model_architecture.predict(n=n_forecast_steps)

# Re-escalar el pronóstico a los valores originales
final_forecast = final_scaler.inverse_transform(final_forecast_scaled)

# --- 3.4 Visualización del Pronóstico Final ---
plt.figure(figsize=(14, 7))
final_series.plot(label='Datos Históricos (Train+Test)')
final_forecast.plot(label='Pronóstico para 2022')
plt.title('Pronóstico Final de Unidades Vendidas para 2022')
plt.xlabel('Fecha')
plt.ylabel('Unidades Vendidas')
plt.legend()
plt.show()


# --- 3.5 Creación del Archivo de Submission ---
print("\nCreando el archivo de submission...")

# Crear un DataFrame con las predicciones
df_forecast = final_forecast.pd_dataframe().reset_index()
df_forecast.columns = ['fecha', 'unidades_vendidas']

# Filtrar las predicciones para que coincidan con las fechas de sample_submission
# que son para el año 2022
df_forecast = df_forecast[df_forecast['fecha'].dt.year == 2022]

# Asegurarse de que el formato de 'unidades_vendidas' sea entero y no negativo
df_forecast['unidades_vendidas'] = df_forecast['unidades_vendidas'].apply(lambda x: max(0, round(x))).astype(int)

# Preparar el archivo de submission final
# sample_submission tiene columnas 'id' y 'unidades_vendidas'
# Suponemos que 'id' se corresponde con las fechas de la semana en 2022
submission = sample_submission.copy()
submission['fecha'] = pd.to_datetime(submission['id']) # Asumimos que 'id' es una fecha

# Unir con nuestras predicciones
submission = pd.merge(submission[['id', 'fecha']], df_forecast, on='fecha', how='left')
submission = submission.drop(columns=['fecha'])

# Llenar posibles valores faltantes (si el pronóstico no cubrió alguna semana)
submission['unidades_vendidas'] = submission['unidades_vendidas'].fillna(0).astype(int)

# Guardar el archivo final
submission.to_csv('submission.csv', index=False)

print("Archivo 'submission.csv' generado exitosamente.")
print("Primeras 5 filas del archivo de submission:")
print(submission.head())

print("\n--- Proceso Finalizado ---")

Librerías importadas correctamente.

--- Iniciando Sección 1: Análisis Exploratorio de Datos ---
Error: No se encontraron los archivos CSV. Por favor, verifica la ruta en la variable 'path'.


NameError: name 'df_train' is not defined

: 